<a id="top"></a>

# Object Detection Demo

## Introduction

This demo shows how a smart video IoT solution may be created using Intel® hardware and software
tools to perform object detection. This solution detects any number of objects within a video frame
looking specifically for known objects.

The results for each frame are stored in a text file that is later read by a second pass to annotate
the input video with boxes around detected objects with a label and probability value.

### Key concepts

This demo includes an example for the following:

- Application:
  - Video and image input is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of the resulting bounding boxes in the output
  - Uses the
    [Async API](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_Intro_to_Performance.html)
    feature of the Inference Engine
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on
    the development node hosting this Jupyter\* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using the
    [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) and
    [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the
    [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)

## Object detection application

The object detection application uses the Intel® Distribution of OpenVINO™ toolkit to perform
inference on an input video to locate known objects within each frame. We will setup, run, and view
the results for this application for several different hardware devices (CPU. GPU, etc.) available
on the compute nodes within the Intel® DevCloud for the Edge. To accomplish this, we will be
performing the following tasks:

1. Use the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html)
   to download the inference model IR files needed to perform inference
2. Use the
   [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
   to create the model IR files in the necessary precisions
3. Create the job file used to submit running inference on compute nodes
4. Submit jobs for different compute nodes and monitor the job status until complete
5. View results and assess performance

### How it works

At startup the object detection application configures itself by parsing the command line arguments
and reading the specified labels file. Once configured, the application loads the specified
inference model's IR files into the
[Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)
and runs the three phases described in the following sections.

#### Phase 1:  Pre-processing

The frames from the specified input video are read and pre-processed to have the frame data ready
for input into inference. Each frame is resized and its channels are transposed to match the input
of the input requirements of the inference model. The resulting blocks of frame data are written to
the binary file `tmp/processed_vid.bin`.

#### Phase 2:  Inference

The binary file `tmp/processed_vid.bin` is read for input to inference to detect known objects. For
each detected object within a frame, a message indicating the object's location, label, etc. is
written as a line to the output text file.

#### Phase 3:  Post-processing

After the object detection application has finished running inference, a second executable is run to
annotate the input video with the results from the output text file. The final results is an output
video with boxes drawn around detected objects with each box appearing with a label and probability
value.

In this sample, we provide three performance numbers for each architecture. However, they are not
hard limits on the solution's performance. It is important to understand that for any application,
you may want to combine preprocessing, inference, and post-processing, as opposed to separating them
as done here. Combined preprocessing has several advantages:

- If inference is run asynchronously on the accelerator, the rest of the system is available for
  parallel tasks for capturing input, preprocessing future frames, or post-processing past frames.
- The inference application pipeline has better data locality, allowing for the reuse of data in
  caches such as in the application's memory or in the hard drive's cache.

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute
node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute
Stick 2. After inference on the input is completed, the output is stored in the appropriate
`results/<JobNum>/` directory. The results are then viewed within this Jupyter\* Notebook using the
`videoHTML` video playback utility.

The application and inference code for this sample is already implemented in the files:

- First pass, detection using inference: [`object_detection.py`](./object_detection.py)
- Second pass, annotation using detection results:
  [`object_detection_annotate.py`](object_detection_annotate.py)

The following sections will guide you through configuring and running the object detection demo.

### Imports

We begin by importing all the Python\* modules that will be used within this Jupyter\* Notebook to
run and display the results of the object detection application on the Intel® DevCloud for the Edge:

- [os](https://docs.python.org/3/library/os.html#module-os) - Operating system specific module (used
  for file name parsing)
- [time](https://docs.python.org/3/library/time.html#module-time) - Time tracking module (used for
  measuring execution time)
- [matplotlib.pyplot](https://matplotlib.org/) - pyplot is used for displaying output images
- [sys](https://docs.python.org/3/library/sys.html#module-sys) - System specific parameters and
  functions
- [qarpo.demoutils](https://github.com/ColfaxResearch/qarpo) - Provides utilities for displaying
  results and managing jobs from within this Jupyter\* Notebook

Run the following cell to import the Python\* dependencies needed.

<br><div class=tip><b>Tip: </b>Select a cell and then use **Ctrl+Enter** to run that cell.</div>

In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
from qarpo.demoutils import liveQstat, progressIndicator, summaryPlot, videoHTML
from qarpo.model_visualizer_link import showModelVisualizerLink

print("Imported Python modules successfully.")

Imported Python modules successfully.


### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the
[Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
used to convert and optimize trained models into the Intermediate Representation (IR) model files,
and the
[Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)
that uses the IR model files to run inference on hardware devices. The IR model files can be created
from trained models from popular frameworks (e.g. Caffe\*, Tensorflow\*, etc.). The Intel®
Distribution of OpenVINO™ toolkit also includes the
[Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) utility to
download some common inference models from the
[Open Model Zoo](https://github.com/opencv/open_model_zoo).

Run the following cell to run the Model Downloader utility with the `--print_all` argument to see
all the available inference models.

In [2]:
!omz_downloader --print_all

Sphereface
aclnet
aclnet-int8
action-recognition-0001
age-gender-recognition-retail-0013
alexnet
anti-spoof-mn3
asl-recognition-0004
background-matting-mobilenetv2
bert-base-ner
bert-large-uncased-whole-word-masking-squad-0001
bert-large-uncased-whole-word-masking-squad-emb-0001
bert-large-uncased-whole-word-masking-squad-int8-0001
bert-small-uncased-whole-word-masking-squad-0001
bert-small-uncased-whole-word-masking-squad-0002
bert-small-uncased-whole-word-masking-squad-emb-int8-0001
bert-small-uncased-whole-word-masking-squad-int8-0002
brain-tumor-segmentation-0001
brain-tumor-segmentation-0002
caffenet
cocosnet
colorization-siggraph
colorization-v2
common-sign-language-0001
common-sign-language-0002
convnext-tiny
ctdet_coco_dlav0_512
ctpn
deblurgan-v2
deeplabv3
densenet-121
densenet-121-tf
detr-resnet50
dla-34
driver-action-recognition-adas-0002
drn-d-38
efficientdet-d0-tf
efficientdet-d1-tf
efficientnet-b0
efficientnet-b0-pytorch
efficientnet-v2-b0
efficientnet-v2-s
emotions-recogn

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning is a special Jupyter\* Notebook command
that allows you to run shell commands as if you are at a command line. The above command will also
work in a terminal (with the '!' removed).</i></div>

Some of these downloaded models are already in the IR format, while others will require the Model
Optimizer to be run. For our application, we will be using the
[`mobilenet-ssd`](https://github.com/opencv/open_model_zoo/tree/master/models/public/mobilenet-ssd)
inference model, which will require being optimization using the Model Optimizer to create the model
in the necessary IR format neede by the Inference Engine to run.

The format for the Model Downloader command to download a model is:

```bash
/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py \
    --name <model_name> -o <output_directory>
```

The input arguments are as follows:

- **--name** : The name of the model you want to download. It should be one of the models listed in
  the previous cell.
- **-o** : The output directory where to store the downloaded model. If the directory does not
  exist, it will be created.

Run the following cell to download the `mobilenet-ssd` model to the `./raw_models` directory
relative to the location of this Jupyter\* Notebook.

In [3]:
#!downloader.py --name mobilenet-ssd -o raw_models
!mkdir -p raw_models/public
!cp -r  /data/reference-sample-data/raw_models/mobilenet-ssd raw_models/public
!echo "\nAll files that were downloaded:"
!find ./raw_models

\nAll files that were downloaded:
./raw_models
./raw_models/public
./raw_models/public/mobilenet-ssd
./raw_models/public/mobilenet-ssd/mobilenet-ssd.prototxt
./raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel


As shown above from the output of the last `!find...` command, the necessary model files created
using the Caffe\* framework have been downloaded:

- **mobilenet-ssd.prototxt** - The deployed
  [Protocol Buffer](https://developers.google.com/protocol-buffers) file that describes the network
  architecture to run inference
- **mobilenet-ssd.caffemodel** - Binary containing trained weights

These files will need to be optimized using the Model Optimizer to create the necessary IR files. We
will be running the inference model on different hardware devices which have different requirements
on the precision of the model (see
[Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats)
for details). For our purposes, we will focus on the use of the two most common precisions, FP32 and
FP16.

For this model, we will run the Model Optimizer using the format:

```bash
/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model <path_to_caffemodel> \
    --data_type <data_precision> \
    --output_dir <path_to_output_directory> \
    --scale <scale_value> \
    --mean_values [<channel_mean_values>] 
```

The input arguments are as follows:

- **--input_model** : The model's input *.caffemodel* file (the *.prototxt* with the same base name
  will be automatically found, otherwise the `--input_proto` argument would need to be specified)
- **--data_type** : The model's data type and precision (e.g. FP16, FP32, INT8, etc.)
- **--output_dir** : Output directory where to store the generated IR model files
- **--scale** : Scaling (divide by) value to apply to input values
- **--mean_values** : Mean values (one per channel) to be subtracted from input values before
  scaling

For converting our model, we will use the typical scaling value `256` and mean values
`[127,127,127]` used with Caffe\* models. The complete command will look like the following:

```bash
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
    --data_type <data_precision> \
    --output_dir models/mobilenet-ssd/<data_precision> \
    --scale 256 \
    --mean_values [127,127,127] 
```

We will run the command twice, first with \<*data_precision*> set to `FP16` and then `FP32` to get
all the IR files we will need to run inference on different devices.

<br><div class=note><i><b>Note: </b>More information on how to use Model Optimizer to convert
Caffe\* models may be found
at:[Converting a Caffe\* Model](https://docs.openvino.ai/latest/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_Caffe.html)</i></div>

Run the following cell to use the Model Optimizer to create the `FP16` and `FP32` model IR files:

In [ ]:
# Create FP16 IR files
!mo \
--input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
--data_type FP16 \
--output_dir models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

# Create FP32 IR files
!mo \
--input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
--data_type FP32 \
--output_dir models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

# find all resulting IR files
!echo "\nAll IR files that were created:"
!find ./models -name "*.xml" -o -name "*.bin" 

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u178245/Reference-samples/iot-devcloud/openvino-dev-latest/developer-samples/python/object-detection-python/raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel
	- Path for generated IR: 	/home/u178245/Reference-samples/iot-devcloud/openvino-dev-latest/developer-samples/python/object-detection-python/models/mobilenet-ssd/FP16
	- IR output name: 	mobilenet-ssd
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	[127,127,127]
	- Scale values: 	Not specified
	- Scale factor: 	256.0
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- 

As shown above from the output of the last `!find...` command, the required sets of IR model files
(`*.xml` and `*.bin`) have been created.

### Optional exercise: View input without inference

If you are curious to see the input video, run the following cell to view the original video stream
used for inference and this sample.

In [ ]:
from qarpo.demoutils import videoHTML

# create link and adjust video path to be able to display from /data using videoHTML()
!ln -sfn /data data
videoHTML(
    "Cars Video", ["./data/reference-sample-data/object-detection-python/cars_1900.mp4"]
)

#### Create the job file

We will run inference on several different edge compute nodes present in the Intel® DevCloud for the
Edge. We will send work to the edge compute nodes by submitting the corresponding non-interactive
jobs into a queue. For each job, we will specify the type of the edge compute server that must be
allocated for the job.

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around
the Python\* executable of our application that will be executed directly on the edge compute node.
One purpose of the job file is to simplify running an application on different compute nodes by
accepting a few arguments and then performing accordingly any necessary steps before and after
running the application executable.

For this sample, the job file we will be using is already written for you and appears in the next
cell. The job file will be submitted as if it were run from the command line using the following
format:

```bash
object_detection_job.sh <output_directory> <device> <fp_precision> <input_file> <num_reqs>
```

Where the job file input arguments are:

- \<*output_directory*> - Output directory to use to store output files
- \<*device*> - Hardware device to use (e.g. CPU, GPU, etc.)
- \<*fp_precision*> - Which floating point precision inference model to use (FP32 or FP16)
- \<*input_file*> - Path to input video file
- \<*num_reqs*> - The maximum number of requests to allow while using the Async API

Based on the input arguments, the job file will do the following:

- Change to the working directory `PBS_O_WORKDIR` where this Jupyter\* Notebook and other files
  appear on the compute node
- Create the \<*output_directory*>
- Choose the appropriate inference model IR file for the specified \<*fp_precision*>
- Run the application Python\* executable with the appropriate command line arguments
- Run the application annotator Python\* executable with the appropriate command line arguments

Run the following cell to create the `object_detection_job.sh` job file. The
[`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile)
line at the top will write the cell contents to the specified job file `object_detection_job.sh`.

In [ ]:
%%writefile object_detection_job.sh

# Store input arguments: <output_directory> <device> <fp_precision> <input_file> <num_reqs>
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_FILE=$4
NUM_REQS=$5

echo VENV_PATH=$VENV_PATH
echo OPENVINO_RUNTIME=$OPENVINO_RUNTIME
echo INPUT_FILE=$INPUT_FILE
echo FP_MODEL=$FP_MODEL
echo INPUT_TILE=$INPUT_FILE
echo NUM_REQS=$NUM_REQS

# Follow this order of setting up environment for openVINO 2022.1.0.553
echo "Activating a Python virtual environment from ${VENV_PATH}..."
source ${VENV_PATH}/bin/activate
echo "Activating OpenVINO variables from ${OPENVINO_RUNTIME}..."
source ${OPENVINO_RUNTIME}/setupvars.sh

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Set inference model IR files using specified precision
MODELPATH=models/mobilenet-ssd/${FP_MODEL}/mobilenet-ssd.xml
LABEL_FILE=./labels.txt

# Run the object detection code
python3 object_detection.py -m $MODELPATH \
                            -i $INPUT_FILE \
                            -o $OUTPUT_FILE \
                            -d $DEVICE \
                            -nireq $NUM_REQS \
                            --labels $LABEL_FILE

# Run the output video annotator code
SCALE_FRAME_RATE=1    # scale number or output frames to input frames
SCALE_RESOLUTION=0.5  # scale output frame resolution 
python3 object_detection_annotate.py -i $INPUT_FILE \
                                     -o $OUTPUT_FILE \
                                     -f $SCALE_FRAME_RATE \
                                     -s $SCALE_RESOLUTION


### Run the demo

Running the next cell will display an interactive user interface allowing you to submit jobs to run
the demo on multiple edge compute nodes selected by hardware devices, view the output of each job,
and compare performance results across jobs.

To run a job:

1. Select the desired option in the **Target node** list
2. Select the desired device in the **Target architecture** list
3. Click the **Submit** button

After the **Submit** button is clicked, a tab will appear for the new job with a label in the format
"*status*: *JobID*". Once the job status appears as "Done", the **Display output** button may be
clicked to view the output for the job.

After one or more jobs are done, the performance results for each job may be plotted by clicking the
**Plot results** button. Results for each job will be potted as bar graphs for **inference time**
and **frames per second**. Lower values mean better performance for **inference time** and higher
values mean better performance for **frames per second**. When comparing results, please keep in
mind that some architectures are optimized for highest performance, others for low power or other
metrics.

Run the next cell to begin the interactive demo.

In [ ]:
# import necessary libraries
import json

import qarpo

# load job configurations for demo
with open("./job_config.json") as json_file:
    data = json.load(json_file)

# create and run the user job interface
job_interface = qarpo.Interface(data)
job_interface.displayUI()

### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command. The custom
Jupyter\* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live
updates.

Run the following cell to display the current job status with periodic updates. You should see the
jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit
the jobs in the previous step). There should also be an extra job in the queue named
`jupyterhub-singleuser`: this job is your current Jupyter\* Notebook session which is always
running.

The `S` column shows the current status of each job:

- If the status is `Q`, then the job is queued and waiting for available resources
- If the status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b> Note: The amount of time spent in the queue depends on the number of
users accessing the requested compute nodes. Once the jobs for this sample application begin to run,
they should take from 1 to 5 minutes each to complete. </b></i></div>

In [ ]:
liveQstat()

# About this notebook

For technical support, please see the
[Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)# Next
steps

- [More Jupyter\* Notebook Samples](https://software.intel.com/content/www/us/en/develop/tools/devcloud/edge/build/sample-apps.html)
  \- additional sample applications
- [Jupyter\* Notebook Tutorials](https://software.intel.com/content/www/us/en/develop/tools/devcloud/edge/learn/tutorials.html)
  \- sample application Jupyter\* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit)
  \- learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for
  implementing inference on the edge

# About this notebook

For technical support, please see the
[Intel® DevCloud Forums](developer-samples/python/shopper-gaze-monitor-python/shopper_gaze_monitor.ipynb)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>